# Comparing effects of DWI Preprocessing and WMQL queries on fiber bundle statistics

## Get Data

10 random INTRuST cases were picked, and WMQL was run under 6 scenarios:

1. Original DWI's and original WMQL query
2. Original DWI's and new WMQL query
3. Harmonized DWI's and original WMQL query
4. Harmonized DWI's and new WMQL query
5. Harmonized/EPI corrected DWI's and original WMQL query
6. Harmonized/EPI corrected DWI's and new WMQL query

(new WMQL query is Nikos/Peter's suggested change to improve e.g ioff fiber bundle results)

In [1]:
load'csv'
localpath=. '/Users/ryan/partners'&,  NB. get local path
download=. ([: shell 'clusterget.sh '&,)`1:@.(fexist@:localpath)
o=.'/data/pnl/intrust-notharmonized/tractmeasures_orig.csv'
on=.'/data/pnl/intrust-notharmonized/wmql-newquery/tractmeasures.csv'
ocpp=.'/data/pnl/intrust-notharmonized-cpp/tractmeasures.csv'
ocppn=.'/data/pnl/intrust-notharmonized-cpp/wmql-newquery/tractmeasures.csv'
h=.'/data/pnl/intrust-harmonized/tractmeasures.csv'
hn=.'/data/pnl/intrust-harmonized/wmql-newquery/tractmeasures.csv'
he=.'/data/pnl/intrust-harmonized-epi/tractmeasures.csv'
hen=.'/data/pnl/intrust-harmonized-epi/wmql-newquery/tractmeasures.csv'
download each REMOTE_PATHS=.o;on;ocpp;ocppn;h;hn;he;hen
csvs=. readcsv@:localpath each REMOTE_PATHS

## Comparison

For each case and for each fiber bundle, we have mean FA, standard deviation of FA,
and number of tracts. Under each of the 6 DWI/WMQL scenarios listed above, we average over the 10 cases and calculate
4 statistics for each bundle:

1. Number of cases for which fiber bundle was found
2. Mean number of tracts (group by 'num_fibers-num_fibers' in tractmeasures.csv)
3. Average mean FA (group by 'FA-mean' in tractmeasures.csv)
4. Average standard deviation of FA (group by 'FA-stDev' in tractmeasures.csv)

In the boxes below, the rows are DWI/WMQL scenarios and the numbers are the 4 statistics.  The
rows are grouped by fiber bundle to compare the effect of each scenario on a fiber bundle's
statistics.

In [3]:
9!:37 [ 0 180 1000 1000
NB. Functions
load'stats regex'
getcol=:}.@:([ {"1~ (i.~ {.)~) 
tonum=. ".every
removeid=: ('^[[:digit:]]{3}_[[:digit:][:alpha:]]{3}_[[:digit:][:alpha:]]{3}_';'')&rxrplc
tractcol=. (removeid each)@:(getcol&(<'tract'))
addcol=: (<@[ ,.~ ])each
has=: +./@:rxE
NB. Get stats
Stats=:/:~@:(~.@:tractcol ,. tractcol <"0@(#,mean)/. tonum@:getcol&('num_fibers-num_fibers';'FA-mean';'FA-stDev')) each csvs
GroupedStats=./:~ ({."1 </. ]) ; (10&}. each REMOTE_PATHS) addcol Stats
Header=: ;: 'Bundle numCases numTractsAvg meanFaAvg stDevFAAvg InputCsv'
Result=. Header , }. ; a:&,each  (#~ 'cb|uf|af|slf|ilf|ioff'&has@:,@:(":@>)@>) GroupedStats
'wmql_comparison.csv' writecsv~ Result
Result

┌─────────────┬────────┬────────────┬─────────┬──────────┬─────────────────────────────────────────────────────────┐
│Bundle       │numCases│numTractsAvg│meanFaAvg│stDevFAAvg│InputCsv                                                 │
├─────────────┼────────┼────────────┼─────────┼──────────┼─────────────────────────────────────────────────────────┤
│af.left      │10      │390.8       │695.219  │167.569   │intrust-notharmonized/tractmeasures_orig.csv             │
├─────────────┼────────┼────────────┼─────────┼──────────┼─────────────────────────────────────────────────────────┤
│af.left      │10      │472.9       │686.361  │167.255   │intrust-notharmonized/wmql-newquery/tractmeasures.csv    │
├─────────────┼────────┼────────────┼─────────┼──────────┼─────────────────────────────────────────────────────────┤
│af.left      │10      │463.7       │649.323  │174.765   │intrust-notharmonized-cpp/tractmeasures.csv              │
├─────────────┼────────┼────────────┼─────────┼──────────┼──────